# Commands

In [1]:
# cd misc-logs ; ./mv-old-logs.sh ; jupython --venv st-venv ~/notebooks/twinews/lda.ipynb

In [2]:
# https://towardsdatascience.com/lets-build-an-article-recommender-using-lda-f22d71b7143e
# https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

# Init

In [3]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [4]:
isNotebook = '__file__' not in locals()

In [5]:
TEST = isNotebook

In [6]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *

In [7]:
from nlptools.topicmodeling import *
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim
from math import log2
from math import sqrt
from numpy import asarray

In [8]:
logger = Logger(tmpDir('logs') + "/lda.log") if isNotebook else Logger("lda.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

# Config

In [40]:
config = \
{
    'maxDocuments': 3000 if TEST else 10000,
    'useExtraNews': False if TEST else True, # None = unlimited, 0 = no extra news
    'maxUsers': 20 if TEST else None, # Sub-sampling
    'nbTopics': 30,
    'lowercase': True,
    'maxIter': 10 if TEST else 30,
    'minDF': 1 / 500, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': 300, # Remove top 300 voc elements
    # <https://www.quora.com/How-do-you-combine-LDA-and-tf-idf>
    # <https://www.quora.com/Why-is-the-performance-improved-by-using-TFIDF-instead-of-bag-of-words-in-LDA-clustering>
    'useTFIDF': False,
    'useSklearn': True,
    'doLemmatization': True if TEST else True,
    'distance': 'cosine', # 'cosine', 'euclidean', 'kl', 'js'
    # The historyRef param is very important, it allow to choose, for a particular candidate,
    # how many train history items will be used to calculate the similarity with
    # the user history.
    # Float are ratio on train history
    # Integers are absolute number of train item in the history
    # For example:
    #  * 1.0 will allow to mean similarities of a candidate with all train history items
    #  * 1 will allow to use only the most similar train item for the similarity of
    #    the candidate with the history of the user
    #  * 0.5 will allow to use the half of history for each candidates
    #  * 3 to use 3 most similar items with the current candidate...
    'historyRef': 1.0, # 1, 1.0, 0.5, 0.3, 3, 10
}

# Getting data

In [11]:
# Getting users and news
evalData = getEvalData(1, maxExtraNews=config['maxDocuments'], maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['stats']), logger)

--> tic: 6.5s | message: Eval data loaded
Unable to create index url in twinews news
twinews news (version 1.0) initialised.
--> tic: 9.05s | message: Extra news downloaded
--> toc total duration: 15.55s | message: Got Twinews evaluation data
{ candidates, created, extraNews, ranksLength, stats, testNews, testUsers, trainNews, trainUsers }
{ testMaxNewsPerUser: 164, testMeanNewsPerUser: 10.67, testMinNewsPerUser: 2, testNewsCount: 138785, totalNewsAvailable: 570210, trainMaxNewsPerUser: 443, trainMeanNewsPerUser: 28.0, trainMinNewsPerUser: 8, trainNewsCount: 323572, usersCount: 21239 }


In [12]:
extraNewsList = shuffle(list(extraNews), seed=0)
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)

In [13]:
# We get urls for the LDA model:
if config['useExtraNews']:
    urlsForModel = extraNewsList + trainNewsList + testNewsList
else:
    urlsForModel = trainNewsList + testNewsList + extraNewsList
urlsForModel = urlsForModel[:config['maxDocuments']]
# We get urls to vectorize for the training and the inference:
urlsToVectorize = copy.deepcopy(urlsForModel)
for url in trainNewsList + testNewsList:
    if url not in urlsToVectorize:
        urlsToVectorize.append(url)
# We get url to infere for the scoring:
urlsToInfere = trainNewsList + testNewsList
# Print all:
log(str(len(urlsForModel)) + " urls for urlsForModel:\n" + b(urlsForModel), logger=logger)
log(str(len(urlsToVectorize)) + " urls for urlsToVectorize:\n" + b(urlsToVectorize), logger=logger)
log(str(len(urlsToInfere)) + " urls for urlsToInfere:\n" + b(urlsToInfere), logger=logger)

3000 urls for urlsForModel:
[
  http://nyti.ms/2AlwYl7,
  https://www.courier-journal.com/story/news/local/2017/12/12/louisville-water-issues-warning-over-wat,
  ...,
  https://fb.me/9mR92sIhm,
  http://wapo.st/2Ecf4GV?tid=ss_tw&utm_term=.b9cd41b923d0
]
19354 urls for urlsToVectorize:
[
  http://nyti.ms/2AlwYl7,
  https://www.courier-journal.com/story/news/local/2017/12/12/louisville-water-issues-warning-over-wat,
  ...,
  https://www.defensenews.com/land/2018/01/17/army-chief-future-of-us-troops-in-iraq-tbd-as-isis-crumb,
  https://www.theatlantic.com/business/archive/2018/02/suburbs-housing/552152
]
19354 urls for urlsToInfere:
[
  http://nyti.ms/2AlwYl7,
  https://www.courier-journal.com/story/news/local/2017/12/12/louisville-water-issues-warning-over-wat,
  ...,
  https://www.defensenews.com/land/2018/01/17/army-chief-future-of-us-troops-in-iraq-tbd-as-isis-crumb,
  https://www.theatlantic.com/business/archive/2018/02/suburbs-housing/552152
]


In [14]:
# We get sentences:
sentences = getNewsSentences(urlsToVectorize, logger=logger)
bp(sentences, logger)

Unable to create index url in twinews news
twinews news (version 1.0) initialised.
  0% [                    ]
  9% [=                   ] (51.851s left)
 19% [===                 ] (43.851s left)
 29% [=====               ] (38.511s left)
 39% [=======             ] (33.266s left)
 49% [=========           ] (27.711s left)
 59% [===========         ] (21.897s left)
 69% [=============       ] (16.592s left)
 79% [===============     ] (10.961s left)
 89% [=================   ] (5.447s left)
 99% [=================== ] (0.011s left)
100% [====================] (total duration: 54.73s, mean duration: 0.002s)
[
  [ [ Last, month, ..., Weinstein, . ], [ Speaking, slowly, ..., myself, . ], ..., [ Accusations, of, ..., paradigm, ) ], [ The, decades, ..., d, . ] ],
  [ [ Live, updates, ..., open, . ], [ Several, streets, ..., streets, . ], ..., [ Matthew, Glowicki, ..., reporting, . ], [ James, Bruggers, ..., today, : ] ],
  ...,
  [ [ U.S., Army, ..., Dec., __int_2__ ], [ (, Spc, . ], ..., 

In [15]:
# We flatten sentences:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ Last, month, ..., d, . ], [ Live, updates, ..., today, : ], ..., [ U.S., Army, ..., Guard, . ], [ Many, neighborhoods, ..., density, . ] ]


In [16]:
# Lower case:
if config['lowercase']:
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

[ [ last, month, ..., d, . ], [ live, updates, ..., today, : ], ..., [ u.s., army, ..., guard, . ], [ many, neighborhoods, ..., density, . ] ]


In [17]:
# Lemmatization:
if config['doLemmatization']:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

Lemmatization   0% [                    ]
Lemmatization   9% [=                   ] (2m 5.308s left)
Lemmatization  19% [===                 ] (1m 35.024s left)
Lemmatization  29% [=====               ] (1m 18.983s left)
Lemmatization  39% [=======             ] (1m 6.082s left)
Lemmatization  49% [=========           ] (54.132s left)
Lemmatization  59% [===========         ] (42.602s left)
Lemmatization  69% [=============       ] (31.727s left)
Lemmatization  79% [===============     ] (20.959s left)
Lemmatization  89% [=================   ] (10.424s left)
Lemmatization  99% [=================== ] (0.021s left)
Lemmatization 100% [====================] (total duration: 1m 43.34s, mean duration: 0.005s)
[ [ last, month, ..., d, . ], [ live, update, ..., today, : ], ..., [ u.s., army, ..., guard, . ], [ many, neighborhood, ..., density, . ] ]


In [18]:
# Filtering the corpus:
docs = filterCorpus(docs, minDF=config['minDF'], maxDF=config['maxDF'],
                    removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
for doc in docs: assert len(doc) > 0
bp(docs, logger)

Voc remove because of minDF (117168 elements):
{ 0, 00s, 01helen, 01st, 02nd, 04th, 05th, 0k, 1, 10, ..., 😄, 😇, 😉, 😍, 😐, 😒, 😓, 😖, 😡, 😱 }
Voc remove because of maxDF (300 elements):
{ ", ', (, ), ,, -, ., :, ;, ?, ..., without, woman, work, working, world, would, year, yet, you, your }
90.49% of voc will be removed.
[ [ access, hollywood, ..., masculinity, d ], [ live, update, ..., journalism, subscribing ], ..., [ army, chief, ..., army, guard ], [ neighborhood, single, ..., increase, density ] ]


# Infering topic vectors

In [19]:
if config['useSklearn']:
    if config['useTFIDF']:
        vectorizer = TfidfVectorizer\
        (
            sublinear_tf=True,
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            # lowercase=True, # Doesn't work because we erased preprocessor
        )
    else:
        vectorizer = CountVectorizer\
        (
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            # lowercase=True, # Doesn't work because we erased preprocessor
        )
    vectors = vectorizer.fit_transform(docs)
    assert vectors.shape[0] == len(urlsToVectorize)
    vectorsForModel = vectors[:len(urlsForModel)]
    assert vectorsForModel.shape[0] == config['maxDocuments']
    i = 0
    for url in urlsToVectorize:
        if url == trainNewsList[0]:
            break
        i += 1
    assert i == len(extraNews) or i == 0
    vectorsForInference = vectors[i:i + len(trainNews) + len(testNews)]
    assert vectorsForInference.shape[0] == len(trainNews) + len(testNews)
    learning_method = 'online'
    learning_offset = 1.0
    lda = LatentDirichletAllocation\
    (
        n_components=config['nbTopics'],
        max_iter=config['maxIter'],
        learning_method=learning_method,
        learning_offset=learning_offset,
        random_state=0,
        n_jobs=cpuCount(),
    )
    lda.fit(vectorsForModel)
    inferedVectors = lda.transform(vectorsForInference)
    assert inferedVectors.shape[0] == len(trainNews) + len(testNews)
    assert inferedVectors[0].shape[0] == config['nbTopics']
    topics = []
    feature_names = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(lda.components_):
        wordProb = []
        for i in range(len(topic)):
            prob = topic[i]
            word = feature_names[i]
            wordProb.append((word, prob))
        wordProb = sortBy(wordProb, desc=True, index=1)[:100]
        current = dict()
        for word, prob in wordProb:
            current[word] = prob
        topics.append(current)
else:
    dictionary = gensim.corpora.Dictionary(docs)
    dictionary.filter_extremes(no_below=config['min_df'])
    bow = [dictionary.doc2bow(doc) for doc in docs]
    if config['useTFIDF']:
        tfidf = gensim.models.TfidfModel(bow)
        bow = tfidf[bow]
    assert len(bow) == len(urlsToVectorize)
    bowForModel = bow[:len(urlsForModel)]
    assert len(bowForModel) == config['maxDocuments']
    i = 0
    for url in urlsToVectorize:
        if url == trainNewsList[0]:
            break
        i += 1
    assert i == len(extraNews) or i == 0
    bowForInference = bow[i:i + len(trainNews) + len(testNews)]
    assert len(bowForInference) == len(trainNews) + len(testNews)
    lda_model = gensim.models.LdaMulticore\
    (
        bowForModel,
        num_topics=config['nbTopics'],
        id2word=dictionary,
        iterations=config['maxIter'],
        workers=cpuCount(),
        passes=3,
    )
    inferedVectors = []
    for current in bowForInference:
        topicProbDistrib = lda_model[current]
        currentVector = [0.0] * config['nbTopics']
        for t, v in topicProbDistrib:
            currentVector[t] = v
        inferedVectors.append(np.array(currentVector))
    assert len(inferedVectors) == len(trainNews) + len(testNews)
    assert len(inferedVectors[0]) == config['nbTopics']
    topics = []
    for i in range(lda_model.num_topics):
        current = dict()
        for x in lda_model.get_topic_terms(i, topn=100):
            current[dictionary[x[0]]] = x[1]
        topics.append(current)

# Showing topics

In [20]:
def printTopics(topics, maxWords=10, logger=None):
    for i in range(len(topics)):
        log(str(i) + ": " + str(" ".join(list(topics[i].keys())[:10])), logger)

In [21]:
def printTopicsOf(vector, topics, logger=None):
    topicsRepr = ""
    topTopics = sortBy([(i, score) for i, score in enumerate(vector) if score > 0.1], desc=True, index=1)[:3]
    log("Top topics number are: " + str(" ".join([str(e[0]) for e in topTopics])), logger)
    currentTopics = [topics[e[0]] for e in topTopics]
    printTopics(currentTopics, logger=logger)

In [22]:
if isNotebook:
    printTopics(topics, logger=logger)

0: rate __float_1__ per market __float_2__ increase average financial income growth
1: project energy industry housing amazon cost building development construction plant
2: event golden bowl january super hosted game won olympics kim
3: election medicaid voter political campaign expansion conservative vote carolina rural
4: technology data design network development information research army defense security
5: game player season coach sport play win field basketball fan
6: louisville kentucky board attorney journal paul photo charge courier federal
7: administration gender men agency feminist ban abortion civil word sex
8: march street car photo hour park saturday road night morning
9: church county drug prison florida marijuana court thomas jail miami
10: study risk brain research researcher data age disease sample water
11: ! love book kind started tell room thought friend kid
12: consumer product store market value industry app focus example standard
13: air force farmer flight sp

# Serialization

In [23]:
if not isNotebook:
    dirPath = nosaveDir() + "/lda-models"
    mkdir(dirPath)
    serialize(lda, dirPath + "/lda1.pickle")
    exit()

# Making a dict url --> topic vector

In [24]:
assert len(urlsToInfere) == len(inferedVectors)
urlsVectors = dict()
for i in range(len(urlsToInfere)):
    urlsVectors[urlsToInfere[i]] = inferedVectors[i]

{
  http://a.msn.com/00/en-ca/AAuP4NT?ocid=st: [5.64294620e-01 3.86751114e-02 8.68055556e-05 2.02162788e-02
   3.98710025e-03 1.56493001e-02 3.46190984e-03 5.37259774e-03
   8.68055556e-05 8.68055556e-05 1.46247216e-02 8.68055556e-05
   1.28142039e-01 2.26685451e-02 8.68055556e-05 8.68055556e-05
   1.04828356e-02 8.68055556e-05 1.94264534e-02 1.01639498e-02
   5.54405679e-02 7.48811254e-03 5.96140302e-02 8.68055556e-05
   8.68055556e-05 8.68055556e-05 1.14881814e-02 7.76197833e-03
   8.68055556e-05 8.68055556e-05],
  http://a.msn.com/00/en-us/AAuRdZ6?ocid=st: [0.12917021 0.23932811 0.00024331 0.00024331 0.00024331 0.00024331
   0.00024331 0.00024331 0.14106146 0.00729253 0.02711513 0.2155088
   0.13545595 0.00024331 0.00024331 0.00024331 0.00024331 0.00024331
   0.00024331 0.00024331 0.00024331 0.01063268 0.01865813 0.00024331
   0.00024331 0.01263781 0.00024331 0.05851633 0.00024331 0.00024331],
  http://a.msn.com/00/en-us/AAuSDKo?ocid=st: [5.13001862e-02 2.48943842e-01 1.74520070e-04

In [25]:
urlsTexts = dict()
allTexts = getNewsText(urlsToInfere, logger=logger)
for i in range(len(urlsToInfere)):
    urlsTexts[urlsToInfere[i]] = allTexts[i]

  0% [                    ]
  9% [=                   ] (50.051s left)
 19% [===                 ] (42.33s left)
 29% [=====               ] (36.83s left)
 39% [=======             ] (31.63s left)
 49% [=========           ] (26.38s left)
 59% [===========         ] (21.037s left)
 69% [=============       ] (15.765s left)
 79% [===============     ] (10.483s left)
 89% [=================   ] (5.233s left)
 99% [=================== ] (0.01s left)
100% [====================] (total duration: 52.04s, mean duration: 0.002s)
{
  http://a.msn.com/00/en-ca/AAuP4NT?ocid=st: The Bank of Canada raised its benchmark interest rate to 1.25 per cent Wednesday and signalled that,,
  http://a.msn.com/00/en-us/AAuRdZ6?ocid=st: Whether you're buying your first or tenth, Jay Leno has a word of warning: Don't let other people sw,
  http://a.msn.com/00/en-us/AAuSDKo?ocid=st: Three weeks after picking up a controversial cargo in the U.K., a liquefied natural gas tanker made ,
  http://a.msn.com/00/en-us/AA

# Similarity function

In [72]:
def getDistances(xvectors, yvectors, metric='cosin', logger=None, verbose=False):
    """
        metric can be 'cosine', 'euclidean', 'kl', 'js'
    """
    # Kullback–Leibler and Jensen-Shannon divergence: 
    def kl_divergence(p, q):
        return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))
    def js_divergence(p, q):
        m = 0.5 * (p + q)
        return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)
    if metric == 'cosine':
        pass
    elif metric == 'euclidean':
        pass
    elif metric == 'kl':
        metric = kl_divergence
    elif metric == 'js':
        metric = js_divergence
    distances = pairwise_distances(xvectors, yvectors, metric=metric)
    return distances

# Prints

In [76]:
if isNotebook:
    userId = list(trainUsers.keys())[11]
    xvectors = []
    xurls = []
    for url in trainUsers[userId]:
        xvectors.append(urlsVectors[url])
        xurls.append(url)
    xvectors = np.array(xvectors)
    yvectors = []
    yurls = []
    for url in candidates[userId][0]:
        yvectors.append(urlsVectors[url])
        yurls.append(url)
    yvectors = np.array(yvectors)
    distances = getDistances(xvectors, yvectors, metric='cosine')

In [77]:
# Printing some docs with topics:
if isNotebook:
    for i in range(10):
        urls = random.choice([xurls, yurls])
        url = random.choice(urls)
        text = urlsTexts[url]
        vector = urlsVectors[url]
        log(url, logger)
        printTopicsOf(vector, topics, logger=logger)
        log(text, logger)
        log("\n" * 2, logger)

https://sports.yahoo.com/derek-jeter-says-people-lost-sight-athletes-kneeling-214331181.html
Top topics number are: 22 5 11
0: republican democrat leader white party washington decision action immigration democratic
1: game player season coach sport play win field basketball fan
2: ! love book kind started tell room thought friend kid
Miami Marlins owner Derek Jeter became one of the few baseball executives to provide his stance on kneeling during the national anthem in October, saying he would be fine with players on his club taking part in the protest.
At a business event Tuesday, Jeter was asked about about the issue again, and laid out some more in-depth thoughts on his stance.
Derek Jeter at the Fast Company Innovation Festival on taking a knee: pic.twitter.com/N4gj16jgOx — Diamond Leung (@diamond83) October 31, 2017
"I don't know what the league rules are, to be quite honest with you, because this is something that just came up last year, so Ive been retired. It's unfortunate ...




https://www.inquisitr.com/4761400/donald-trump-called-traitor-accused-of-conspiring-with-russia-by-refusing-to-enact-sanctions/
Top topics number are: 15 22
0: campaign russian russia medium google intelligence fbi memo committee clinton
1: republican democrat leader white party washington decision action immigration democratic
The sanctions had passed with overwhelming majorities in the House and Senate, but the State Department said they will not be put in place.
Donald Trump is being called a "traitor" and accused of conspiring with Russia after the president refused to enact sanctions that had been passed with overwhelming majorities in the House and Senate.
Last year, Trump had reluctantly signed the bill creating new sanctions on Russia for the country's interference in the 2016 presidential election. Reports from the intelligence community found that Russia purposely meddled in the election in the hope of helping Trump get elected, and a salacious dossier compiled by a former




https://www.washingtonpost.com/politics/john-kellys-credibility-is-at-risk-after-defending-aide-accused-of-domestic-violence/2018/02/08/e8e1ff06-0ccf-11e8-8890-372e2047c935_story.html
Top topics number are: 22 24 28
0: republican democrat leader white party washington decision action immigration democratic
1: black white girl film music movie moore man johnson character
2: mr. sexual allegation court judge lawyer harassment statement assault claim
White House Chief of Staff John F. Kelly first found his credibility being challenged in October, when he leveraged his standing as a retired four-star Marine Corps general who had lost a son on the battlefield to try to contain a political crisis over President Trump's calls to the families of fallen soldiers.
His reputation took another hit when he later refused to apologize for falsely attacking a Democratic congresswoman. And another when he called Confederate Gen. Robert E. Lee "honorable" and blamed the Civil War on a lack of comprom




https://medium.com/umass-zoodisc/12-z018-75dfa47f586
Top topics number are: 8 11 5
0: march street car photo hour park saturday road night morning
1: ! love book kind started tell room thought friend kid
2: game player season coach sport play win field basketball fan
Zoo Spring Season Starts This Weekend!!
The UMass A Team heads to Charlotte, North Carolina this weekend for the first tournament of 2018: Queen City Tune-Up. Follow along on Twitter @UmassUltimateM or @ultiworldlive
Tiina and I have launched a series on Ultiworld. It will "offer a template for coaches (or captains) on how to put together a new team or how to help a struggling team." Follow the link to Ultiworld to learn more, and follow along with that series throughout the spring.
The amazing team over at VC Ultimate has created the first ever UMASS Men's Ultimate Gear Store. Follow the link to find a selection of ten items that are on sale for a limited time.
One of ten items for sale in our team store
Winter Break T

In [78]:
# Printing some similar docs:
if isNotebook:
    for i in range(distances.shape[0]):
        if i > 100:
            break
        # Get train:
        trainUrl = xurls[i]
        trainText = urlsTexts[trainUrl]
        trainVector = urlsVectors[trainUrl]
        log(trainUrl, logger)
        printTopicsOf(trainVector, topics, logger=logger)
        log(trainText[:2000], logger)
        log("\n", logger)
        # Get distances:
        currentDistances = []
        for u in range(len(yurls)):
            currentDistances.append((yurls[u], urlsTexts[yurls[u]], distances[i][u]))
        topSim = sortBy(currentDistances, index=2, desc=False)[:3]
        topDissim = sortBy(currentDistances, index=2, desc=True)[:3]
        # Print similars:
        log("MOST SIMILARS", logger)
        log("\n", logger)
        for url, text, dist in topSim:
            log(dist, logger)
            log(url, logger)
            printTopicsOf(urlsVectors[url], topics, logger=logger)
            log(text[:2000], logger)
            log("\n", logger)
        # Print dissimilars:
        log("MOST DISSIMILARS", logger)
        log("\n", logger)
        for url, text, dist in topDissim:
            log(dist, logger)
            log(url, logger)
            printTopicsOf(urlsVectors[url], topics, logger=logger)
            log(text[:2000], logger)
            log("\n", logger)
        log("\n", logger)
        log("\n" * 2 + '-' * 20 + "\n" * 2, logger)        

https://www.nbcnews.com/news/us-news/amp/trump-signs-bill-revoking-obama-era-gun-checks-people-mental-n727221?__twitter_impression=true
Top topics number are: 22 29
0: republican democrat leader white party washington decision action immigration democratic
1: police gun officer shooting la crime vega violence victim shot
Trump Signs Bill Revoking Obama-Era Gun Checks for People With Mental Illnesses
Trump Signs Bill Revoking Obama-Era Gun Checks for People With Mental Illnesses
by Ali Vitali /
Mar.01.2017 / 1:39 AM ET
A clerk helps a customer shop for a handgun in Illinois in 2014.Scott Olson / Getty Images, file
Breaking News Emails
Get breaking news alerts and special reports. The news and stories that matter, delivered weekday mornings.
President Donald Trump quietly signed a bill into law Tuesday rolling back an Obama-era regulation that made it harder for people with mental illnesses to purchase a gun.
The rule, which was finalized in December, added people receiving Social Securi



MOST DISSIMILARS


0.9973345982335142
https://goo.gl/bDq6Ev
Top topics number are: 1 0 4
0: project energy industry housing amazon cost building development construction plant
1: rate __float_1__ per market __float_2__ increase average financial income growth
2: technology data design network development information research army defense security
According to information released by Statistics Estonia on Wednesday, the production of industrial businesses rose eight percent on year in 2017, with production volumes increasing in energy, mining and manufacturing.
In 2017, total production in manufacturing was seven percent higher than in 2016.
In more than two thirds of the branches of industry, production surpassed the volume of the previous year. The growth in production was due primarily to an increase in the manufacture of wood products, food products, metal products and electrical equipment. Among branches of industry with larger shares, production fell in the manufacture of electr



MOST SIMILARS


0.061696508617395174
http://wapo.st/2BVcAYh
Top topics number are: 28 11 22
0: mr. sexual allegation court judge lawyer harassment statement assault claim
1: ! love book kind started tell room thought friend kid
2: republican democrat leader white party washington decision action immigration democratic
She watched her son leave for high school, flipped on the flat screen and then, Alecia McNair waited. Most days, her habit was to listen absent-mindedly to "CBS This Morning" as she prepared for another day as a 53-year-old single mother. But this past fall, it seemed that every time she turned on the news, there was another story about a man who had abused his power, a woman who had finally spoken up, and — this was the part that McNair could hardly believe — an apology.
"And that was something I feel that I never got, " she says. She was like so many women, carrying around an experience that she had never really talked about, that she blamed herself for, that she had 



0.9969798410517917
https://buff.ly/2EnvrgK
Top topics number are: 0 12
0: rate __float_1__ per market __float_2__ increase average financial income growth
1: consumer product store market value industry app focus example standard
or an early-stage biotech company, raising venture capital funding for the first time is a crucial milestone. And when you look at all the baby's-first-financing announcements of the past few years, there's something going on that Atlas Venture VC Bruce Booth calls "so counterintuitive in a lot of ways."
New data show that the number of life sciences companies that reach that pivotal moment has stayed flat over the past few years. But the amount of money being invested in these first financings is rising, indicating that when companies do manage to get a first big investment, they're getting more money. The figures come from PitchBook, a company that analyzes and sells data on the private markets.


0.9964646508755818
http://www.islandpacket.com/news/local/c



0.04841453144868213
http://ow.ly/D3cy30i0ohH
Top topics number are: 20 22 12
0: tax bill budget republican care cut insurance congress senate spending
1: republican democrat leader white party washington decision action immigration democratic
2: consumer product store market value industry app focus example standard
The Affordable Care Act helped dramatically reduce the number of people who lack health coverage in New Jersey, but those who signed up are least satisfied with their coverage because of cost, according to a poll released Monday.
People who bought their coverage through the Obamacare health exchange were the most likely to say cost drove their decision -- not because they liked the network of doctors or they wanted to maintain the same coverage from the prior year, according to the findings by the Eagleton Center for Public Interest Polling and the New Jersey Health Care Quality Institute.
"This group is the least satisfied about current costs and thus considers cost most

...



MOST DISSIMILARS


0.9977672281883181
http://bit.ly/2nkc2a3
Top topics number are: 27 16 4
0: social experience using user digital video content medium model phone
1: student education teacher college district campus class parent graduate skill
2: technology data design network development information research army defense security
AI superstar Andrew Ng has spent the past year launching a handful of new initiatives, and now he's at it again with the AI Fund, a startup incubator that will back small teams of experts looking to solve key problems using machine learning.
The fund has $175 million in its war chest to fund AI startups that arise within its walls. Rather than bring in outside startups and provide them with funding, Ng's vision is to have small teams working on projects that can evolve into larger companies. It's similar to what he did at Baidu, where he led the Chinese search giant's artificial intelligence efforts.
Ng will serve as the incubator's general partner and lea



MOST SIMILARS


0.017511969589253162
https://www.dallasnews.com/news/politics/2018/01/22/trumps-voter-fraud-commission-requested-texasrecords-identified-hispanic-voters
Top topics number are: 22 3 28
0: republican democrat leader white party washington decision action immigration democratic
1: election medicaid voter political campaign expansion conservative vote carolina rural
2: mr. sexual allegation court judge lawyer harassment statement assault claim
Get Unlimited Digital Access Your first month is less than a dollar. $0.99 for first 4 weeks Subscribe Now
AUSTIN — President Donald Trump's now disbanded voter fraud commission sought voter records from Texas state officials that flagged Hispanic voters, The Washington Post reported Monday .
The commission's request seeking categories such as full names, addresses, dates of birth, political party affiliation, last four digits of Social Security numbers and voter history since 2006 had rattled voting rights groups in Texas, who fear



0.9929462090547879
https://goo.gl/bDq6Ev
Top topics number are: 1 0 4
0: project energy industry housing amazon cost building development construction plant
1: rate __float_1__ per market __float_2__ increase average financial income growth
2: technology data design network development information research army defense security
According to information released by Statistics Estonia on Wednesday, the production of industrial businesses rose eight percent on year in 2017, with production volumes increasing in energy, mining and manufacturing.
In 2017, total production in manufacturing was seven percent higher than in 2016.
In more than two thirds of the branches of industry, production surpassed the volume of the previous year. The growth in production was due primarily to an increase in the manufacture of wood products, food products, metal products and electrical equipment. Among branches of industry with larger shares, production fell in the manufacture of electronic equipment and 



0.13821480037311618
http://heatzone.blog.palmbeachpost.com/2018/02/08/dwyane-wade-is-coming-home-heat-acquiring-greatest-player-in-franchise-history-from-cleveland-for-second-round-pick/
Top topics number are: 5 11
0: game player season coach sport play win field basketball fan
1: ! love book kind started tell room thought friend kid
[Dwyane Wade: By the numbers]
"I guess I can share my city with him again, " said Udonis Haslem, Wade's closest friend on the team who said last summer that if Wade ever returned, it would be "a dream come true."
Wade arrived in Miami on Thursday night and he is expected to play in Friday's home game against Milwaukee. Heat President Pat Riley, who said the trade came together quickly, talked to Wade after the deal was made.
"I said to him, 'You just ate your last meal of pancakes with syrup and with whip cream and strawberries on top. We're back to veggies.' " Riley said.
The Heat had to create a roster spot for Wade and released center A.J. Hammons, wh



0.996421263245387
http://www.foxnews.com/us/2018/02/14/shots-fired-outside-national-security-agency-suspect-in-custody-reports-say.html
Top topics number are: 29 18 8
0: police gun officer shooting la crime vega violence victim shot
1: vehicle training china coal technology industry military pollution chemical paris
2: march street car photo hour park saturday road night morning
Three people were taken into custody Wednesday morning after a black SUV attempted to enter the National Security Agency campus in Maryland without authorization. The incident ended with the bullet-riddled vehicle being stopped by barriers outside the gate, officials said.
The three taken into custody were passengers in the SUV when it attempted to make its way onto the NSA compound just after 7 a.m., Gordon Johnson, Special Agent in Charge at the FBI Baltimore Office, said in an afternoon news conference.
The SUV, which had a New York license plate, appeared to be a rental car, Johnson said. He added that th

'730462767191232512'

# Ranking

In [79]:
historyRef = config['historyRef']
assert (isinstance(historyRef, int) and historyRef >= 1) or (isinstance(historyRef, float) and historyRef > 0.0 and historyRef <= 1.0)

In [80]:
if isNotebook:
    historyRef = 1.0

In [81]:
ranks = dict()
for userId in trainUsers.keys():
    if isinstance(historyRef, float):
        currentHistoryRef = int(historyRef * len(xurls))
    else:
        currentHistoryRef = historyRef
    xvectors = []
    xurls = []
    for url in trainUsers[userId]:
        xvectors.append(urlsVectors[url])
        xurls.append(url)
    xvectors = np.array(xvectors)
    ranks[userId] = []
    for currentCandidates in candidates[userId]:
        yvectors = []
        yurls = []
        for url in currentCandidates:
            yvectors.append(urlsVectors[url])
            yurls.append(url)
        yvectors = np.array(yvectors)
        distances = getDistances(xvectors, yvectors, metric=config['distance'], logger=logger)
        urlDistances = dict()
        for testIndex in range(len(yurls)):
            url = yurls[testIndex]
            currentDists = distances[:, testIndex]
            assert currentDists.shape[0] == len(xurls)
            currentDists = sorted(list(currentDists), reverse=False)
            currentDists = currentDists[:currentHistoryRef]
            currentDist = np.mean(currentDists)
            urlDistances[url] = currentDist
        rank = [e[0] for e in sortBy(urlDistances, index=1, desc=False)]
        ranks[userId].append(rank)

In [82]:
bp(ranks, logger, 4)

{
  '1315439827': 
  [
    [
      https://thequirkyladybookworm.wordpress.com/2018/01/21/spotlight-on-pachinko/,
      http://www.dorsetecho.co.uk/news/15913597.dippy-the-diplodocus-has-officially-arrived-at-dorset-county-museum/,
      http://ow.ly/Yx2630hX60v,
      https://www.chronicle.com/article/History-in-the-Face-of/242401/,
      https://medium.com/words-for-life/8-airplane-assholes-who-are-worse-seatmates-than-my-toddler-1bbcffff1f31,
      https://www.theguardian.com/lifeandstyle/2018/jan/18/feeling-lonely-meet-people-extreme-isolation-found-happiness,
      https://m.huffpost.com/us/entry/us_570f1a96e4b03d8b7b9f7e92,
      https://fb.me/1vdzvUNH0,
      https://www.irishtimes.com/culture/books/irish-women-who-rocked-the-system-1.3377967,
      http://www.kansas.com/living/family/article194877429.html,
      ...,
      https://uk.reuters.com/article/uk-opec-oil/opec-sees-higher-2018-oil-demand-but-more-rival-supply-idUKKBN1FW14O,
      http://wapo.st/2n59lIt?tid=ss_tw&utm_t